In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from typing import List, Optional
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from src.utils.file_utils import convert_time_to_seconds, create_full_date

In [3]:
conn = sqlite3.connect("data/athle_results.sqlite")
try:
    df = pd.read_sql_query(f"SELECT * FROM results", conn)
except Exception as e:
    print(f"Erreur lors de la lecture de la base : {e}")

In [4]:
qp_800 = df[(df.seq == "49504851445551484851475246535148") & ((df.Epreuve == "800m") | (df.Epreuve == "800m Piste Courte"))]

In [5]:
qp_800

,seq,Club,Date,Epreuve,Tour,Pl.,Perf.,Vt.,Niv.,Pts,Ville,Annee
683,49504851445551484851475246535148,92111.0,06/10,800m,Final 1,5,2'02''99,,IR4,689.0,Aulnay Sous Bois,2024
688,49504851445551484851475246535148,92111.0,22/06,800m,Final 5,5,2'01''01,,IR4,736.0,Viry Chatillon,2024
691,49504851445551484851475246535148,92111.0,20/05,800m,Final 1,3,1'57''42,,IR2,825.0,Lille,2024
729,49504851445551484851475246535148,38072.0,03/10,800m,Final 1,6.0,2'03''24,,R1,683.0,Bonneville,2021
730,49504851445551484851475246535148,38072.0,26/09,800m,Final 1,6.0,2'02''89,,IR4,691.0,Grenoble,2021
733,49504851445551484851475246535148,38072.0,01/07,800m,Final 6,2.0,2'02''64,,IR4,697.0,Bron,2021


In [6]:
# Appliquer la fonction pour créer la colonne full_date
qp_800['full_date'] = qp_800.apply(create_full_date, axis=1)
qp_800['full_date'] = pd.to_datetime(qp_800['full_date'], format='%d/%m/%Y')

C:\Users\RAGNAR\AppData\Local\Temp\ipykernel_30636\2125332720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qp_800['full_date'] = qp_800.apply(create_full_date, axis=1)
C:\Users\RAGNAR\AppData\Local\Temp\ipykernel_30636\2125332720.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qp_800['full_date'] = pd.to_datetime(qp_800['full_date'], format='%d/%m/%Y')


In [9]:
# Création d'une variable Lieu
qp_800['Lieu'] = np.where(qp_800.Epreuve == "800m Piste Courte", 'Indoor', 'Outdoor')

# Création d'une variable Annee
qp_800['Annee'] = qp_800['full_date'].dt.year 

# Suppression des DFN et DNS
qp_800 = qp_800[~qp_800['Perf.'].str.contains('|'.join(['DNS','DNF', 'AB']))]

# Conversion des perf en secondes
qp_800['time'] = qp_800['Perf.'].apply(convert_time_to_seconds)

# Calculating the best performance per year
best_performances = qp_800[['Perf.','time','full_date','Annee']].sort_values('time', ascending=True).groupby('Annee').first().reset_index()
best_performances

C:\Users\RAGNAR\AppData\Local\Temp\ipykernel_30636\3333539511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qp_800['Lieu'] = np.where(qp_800.Epreuve == "800m Piste Courte", 'Indoor', 'Outdoor')
C:\Users\RAGNAR\AppData\Local\Temp\ipykernel_30636\3333539511.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qp_800['Annee'] = qp_800['full_date'].dt.year


,Annee,Perf.,time,full_date
0,2021,2'02''64,122.64,2021-07-01
1,2024,1'57''42,117.42,2024-05-20


In [16]:
# # This code snippet is customizing and creating a scatter plot visualization using the Seaborn library in Python. Here is a breakdown of what the code is doing:
# # Paramètres de personnalisation
# sns.set_style("whitegrid")
# medium_colors = [
#     "#6A0572", "#9A1672", "#E8515A", "#FFA500", 
#     "#FFC947", "#00A878", "#008DD5", "#6E7BA8", 
#     "#9C82B8", "#F08CA5", "#4CBB17", "#FF6B6B"
# ]
# # Conversion des valeurs de l'axe des ordonnées en minutes, secondes et centièmes
# def format_time(x):
#     minutes = int(x) // 60
#     seconds = int(x) % 60
#     centiseconds = int((x * 100) %100)
#     return f"{minutes}'{seconds:02d}''{centiseconds:02d}"

# # Créer une figure avec des sous-graphiques (axes)
# fig, ax = plt.subplots(figsize=(12, 6), dpi=300)

# # Tracer le scatterplot avec la couleur des meilleures performances
# scatter_plot = sns.scatterplot(data=qp_800, x='full_date', y='time', hue='Annee', style='Lieu',
#                  s=70, palette=medium_colors, ax=ax)

# # Ajouter les annotations pour les meilleures performances devant les points
# for index, row in best_performances.iterrows():
#     time_format = format_time(row['time'])
#     ax.scatter(row['full_date'], row['time'], color='red', marker='s', s=100)
#     ax.text(row['full_date'], row['time'], f"{time_format}", ha='center', va='top', fontsize=10, fontweight='bold')

# # Personnaliser les axes
# ax.set_xlabel("Année", fontweight='bold')
# ax.set_ylabel("Performance", fontweight='bold')
# ax.set_title(f"Evolution des performances de Q. Pottier sur 800m", fontweight='bold')

# # Conversion de l'axe des ordonnées en minutes, secondes et centièmes
# ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: format_time(x)))

# # Obtenir les étiquettes distinctes des années pour la légende
# legend_labels = qp_800['Annee'].unique()

# # Obtenir les handles et labels de la légende avec le style du scatterplot
# handles, labels = scatter_plot.get_legend_handles_labels()

# # Ajouter le symbole et la légende pour la meilleure performance par année
# best_handle = plt.Line2D([], [], marker='s', color='red', markersize=10, markerfacecolor='red', markeredgewidth=1.5)
# handles = [best_handle] + handles
# labels = ['Best'] + labels

# # Afficher la légende en dehors des axes (à droite)
# ax.legend(handles=handles, labels=labels, title="Best Per Year", loc='center left', bbox_to_anchor=(1, 0.5))

# # Afficher la grille
# ax.grid(True)

# # Ajuster les marges
# plt.tight_layout()

# # Afficher le graphique
# plt.show()

In [104]:
conn.close()

In [98]:
conn = sqlite3.connect("data/athle_results.sqlite")
try:
    df = pd.read_sql_query(f"SELECT * FROM athletes", conn)
except Exception as e:
    print(f"Erreur lors de la lecture de la base : {e}")

In [99]:
df

,seq,name,club,sex
0,4851475245544851495050494554,VAUDOIS Aurelien,- Athle 92*,M
1,495046534653504950494257,DIRAT Franck,- Athle 92*,M
2,49504851445551484851475246535148,POTTIER Quentin,- Athle 92*,M
